In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install transformers
!pip install torch

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/toxic_sentence_merged.csv')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments


# 사전 학습 모델 로드 (2진 분류를 위한 출력층 추가)
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-small-uncased", num_labels=2)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-small-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-small-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer
import torch

# 텍스트를 토큰화
inputs = tokenizer(list(data['sentence']), padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor(data['label'].values)  # 레이블을 텐서로 변환

In [ ]:
from transformers import AutoModelForSequenceClassification

# 모델 로드 (2진 분류 작업을 위한 출력층 설정)
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-small-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-small-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

# Config 생성 및 수정
config = AutoConfig.from_pretrained("nlpaueb/legal-bert-small-uncased", num_labels=2)
config.max_position_embeddings = 1024  # 수정

# 모델 생성
model = AutoModelForSequenceClassification.from_config(config)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/toxic_sentence_merged.csv')
data_temp = data.copy()

In [ ]:
from torch.utils.data import Dataset
import torch

# Dataset 클래스를 상속하여 커스터마이징
class ContractDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # 텍스트를 토큰화
        encoding = self.tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=1024, return_tensors='pt')

        # 토큰화된 텍스트와 레이블을 반환 (튜플 형식으로)
        item = {key: encoding[key].squeeze(0) for key in encoding}  # 차원 축소
        item['labels'] = torch.tensor(self.labels[idx])  # 레이블 추가
        return item

In [ ]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import random_split

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',          # 결과 디렉토리
    num_train_epochs=30,              # 훈련 epoch 수
    per_device_train_batch_size=8,   # 배치 사이즈
    learning_rate=5e-5,
    eval_strategy="epoch",     # 평가 주기
    logging_dir='./logs',            # 로그 디렉토리
)

# Custom Dataset 객체 생성
train_dataset = ContractDataset(data_temp['sentence'].tolist(), data_temp['label'].tolist(), tokenizer)

train_size = int(0.7 * len(train_dataset))  # 70% 훈련
val_size = int(0.2 * len(train_dataset))    # 20% 검증
test_size = len(train_dataset) - train_size - val_size  # 나머지 10% 테스트

# 데이터셋을 훈련, 검증, 테스트로 나누기
train_dataset, eval_dataset, test_dataset = random_split(train_dataset, [train_size, val_size, test_size])

# Trainer에 Dataset 객체 전달
trainer = Trainer(
    model=model,                         # 모델
    args=training_args,                  # 훈련 인자
    train_dataset=train_dataset,         # 훈련 데이터셋
    eval_dataset=eval_dataset,          # 평가 데이터셋 (선택적으로 설정)
)

# 훈련 시작
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.670757
2,No log,0.695728
3,No log,0.658310
4,No log,0.678955
5,No log,0.646059
6,0.604300,0.598098
7,0.604300,0.606020
8,0.604300,0.669029
9,0.604300,0.601090
10,0.604300,0.629489


TrainOutput(global_step=2910, training_loss=0.4448265521387054, metrics={'train_runtime': 1531.4323, 'train_samples_per_second': 15.084, 'train_steps_per_second': 1.9, 'total_flos': 2722007844864000.0, 'train_loss': 0.4448265521387054, 'epoch': 30.0})

In [ ]:
# 모델 로드
# wandb key : e06e8b1c2f26095018e41310344bfdafc2e1af80

In [ ]:
# 평가
results = trainer.evaluate()

print("Evaluation results: ", results)

Evaluation results:  {'eval_loss': 0.9445316195487976, 'eval_runtime': 3.5819, 'eval_samples_per_second': 61.42, 'eval_steps_per_second': 7.817, 'epoch': 30.0}


In [ ]:
data_temp['predicted_label_percent'] = None  # 초기 값으로 None 또는 다른 값을 할당
# NaN으로 초기화
data_temp['predicted_label'] = np.nan

In [ ]:
# 예측 수행 (테스트 데이터셋을 기준으로)
predictions, label_ids, metrics = trainer.predict(test_dataset)

# 로짓을 확률로 변환 (Sigmoid 적용)
probabilities = torch.sigmoid(torch.tensor(predictions))  # 로짓을 확률로 변환

# 양성 클래스(1번 클래스)의 확률을 선택 (1D 배열로 변환)
predicted_label_percent = probabilities[:, 1].numpy()  # 확률 값 (1D 배열)

# 예측된 레이블 추출 (0 또는 1)
predicted_labels = (predicted_label_percent > 0.5).astype(int)  # 0.5를 기준으로 레이블 결정

# 예측 결과를 data_temp에 해당하는 부분에 추가
start_index = len(train_dataset) + len(eval_dataset)
end_index = start_index + len(test_dataset)

# 'predicted_label' 컬럼 초기화
data_temp['predicted_label'] = np.nan  # NaN으로 초기화

# 예측된 레이블을 'predicted_label' 컬럼에 추가
data_temp.iloc[start_index:end_index, data_temp.columns.get_loc('predicted_label')] = predicted_labels

# 예측된 확률 값 (predicted_label_percent)을 'predicted_label_percent' 컬럼에 추가
data_temp.loc[start_index:end_index-1, 'predicted_label_percent'] = predicted_label_percent

In [ ]:
data_temp

,sentence,label,predicted_label_percent,predicted_label
0,"본 계약에 따라 공급된 상품에 대해 모든 책임은 을이 단독으로 부담하며, 갑은 어떠...",1,None,NaN
1,본 계약에 따라 납품된 상품에 대한 환불 및 반품 요청은 어떠한 경우에도 불가하다.,1,None,NaN
2,"을은 갑의 요청이 있을 경우, 별도의 대가 없이 추가 물량을 무상으로 공급해야 한다.",1,None,NaN
3,"갑은 언제든지 을의 동의 없이 계약을 해지할 수 있으며, 을은 이에 대해 어떠한 이...",1,None,NaN
4,"을이 계약을 위반할 경우 즉시 손해배상 책임을 지며, 갑은 이에 대한 구체적인 증빙...",1,None,NaN
...,...,...,...,...
1096,"③ ""갑""과 ""을""이 2023년 7월 5일에 따른 조정을 신청하지 아니하는 경우, ...",1,0.646392,1.0
1097,※ 중재에 의해 분쟁을 최종적으로 해결하고자 하는 경우에는 2022년 2월 28일자...,1,0.932636,1.0
1098,"③ ""갑""과 ""을""이 2023년 10월 31일까지 제2항에 의한 조정을 신청하지 아...",1,0.903633,1.0
1099,중재기관: 서울중앙지방법원(2024년 5월 10일 기준으로 추가 수수료 15%를 요...,1,0.680467,1.0


In [ ]:
# 1. 'label'과 'predicted_label'이 일치하는 행 필터링 (test_dataset에 해당하는 부분)
matched_data = data_temp.iloc[len(train_dataset) + len(eval_dataset):][data_temp['label'] == data_temp['predicted_label']]
unmatched_data = data_temp.iloc[len(train_dataset) + len(eval_dataset):][data_temp['label'] != data_temp['predicted_label']]

# 2. 일치하는 확률 계산
match_probability = len(matched_data) / len(test_dataset)
unmatch_probability = len(unmatched_data) / len(test_dataset)

# 일치 확률 출력
print(f"일치 확률: {match_probability * 100:.2f}%")
print(f"일치하지 않을 확률: {unmatch_probability * 100:.2f}%")

일치 확률: 54.95%
일치하지 않을 확률: 45.05%


<ipython-input-22-92451c0603f6>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matched_data = data_temp.iloc[len(train_dataset) + len(eval_dataset):][data_temp['label'] == data_temp['predicted_label']]
<ipython-input-22-92451c0603f6>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unmatched_data = data_temp.iloc[len(train_dataset) + len(eval_dataset):][data_temp['label'] != data_temp['predicted_label']]


In [ ]:
# 'predicted_label_percent' 컬럼에서 NaN이 아닌 값만 추출
filtered_data = data_temp[data_temp['predicted_label_percent'].notna()]

In [ ]:
# filtered_data를 CSV로 저장
filtered_data.to_csv('legal-bert-small_ver1.csv', index=False, encoding='utf-8-sig')

In [ ]:
filtered_data

,sentence,label,predicted_label_percent,predicted_label
697,초과판매액의 50%에 해당하는 금액은 갑이 을에게 2023년 5월 1일까지 지급해야...,1,0.941583,1.0
698,"**문장:** \n(1) 본 계약은 2023년 4월 1일부터 효력이 발생하며, 계...",1,0.246354,0.0
699,② 제1항의 판매장려금률(액)의 결정기준ㆍ결정절차 및 변경사유ㆍ변경기준ㆍ변경절차는 ...,1,0.942036,1.0
700,**문장:**\n1. 결정기준: 이 계약에 의해 제공되는 서비스의 수행과 관련하여 ...,1,0.566047,1.0
701,**원본 문장:**\n2. 결정절차:\n\n**수정된 문장:**\n2. 결정절차:\...,1,0.459984,0.0
...,...,...,...,...
770,"③ ""갑""과 ""을""이 제2항에 따른 조정을 신청하지 아니하는 경우, 이 계약에 관한...",1,0.945011,1.0
771,※ 중재에 의해 분쟁을 최종적으로 해결하고자 하는 경우에는 제3항을 다음과 같이 규...,1,0.566047,1.0
772,"③ ""갑""과 ""을""이 제2항에 의한 조정을 신청하지 아니하는 경우, 다음의 중재기관...",1,0.459187,0.0
773,**중재기관:** 갑과 을 사이에 발생한 분쟁은 을이 지정하는 중재기관에 의해 해결...,1,0.44476,0.0
